In [1]:
import os

In [2]:
%pwd

'd:\\projects\\Data-Extraction-and-NLP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\projects\\Data-Extraction-and-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.DataExtractionAndNLP.constants import *
from src.DataExtractionAndNLP.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        # schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
from src.DataExtractionAndNLP import logger

In [9]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict
import re
from src.DataExtractionAndNLP.utils.common import get_name

In [10]:
class PreCleaning:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def calculate(self, data):
        filename = get_name(data)

        output_file = os.path.join(self.config.root_dir, filename)

        with open(output_file, 'r', encoding='utf-8') as file:
            text = file.read()

            # Function to calculate average word length
            def average_word_length(text):
                words = word_tokenize(text)
                total_chars = sum(len(word) for word in words)
                num_words = len(words)
                return total_chars / num_words
            
            # Function to count syllables in a word
            def count_syllables(word):
                vowels = 'aeiouAEIOU'
                syllables = 0
                prev_char_vowel = False
                for char in word:
                    if char in vowels:
                        if not prev_char_vowel:
                            syllables += 1
                        prev_char_vowel = True
                    else:
                        prev_char_vowel = False
                return syllables
            
            # Function to count complex words (more than two syllables)
            def count_complex_words(text):
                words = word_tokenize(text)
                d = cmudict.dict()
                complex_words = [word for word in words if len(d.get(word.lower(), [])) > 2]
                return len(complex_words)
            
            # Function to calculate average sentence length
            def average_sentence_length(text):
                sentences = sent_tokenize(text)
                num_sentences = len(sentences)
                words = word_tokenize(text)
                num_words = len(words)
                return num_words / num_sentences

            # Calculate metrics
            avg_sentence_length = average_sentence_length(text)
            complex_word_count = count_complex_words(text)
            syllable_count = sum(count_syllables(word) for word in word_tokenize(text))
            personal_pronouns_count = len(re.findall(r'\b(I|we|my|ours)\b', text, flags=re.IGNORECASE))
            personal_pronouns_count+=len(re.findall(r'\b(us|Us|uS)\b', text))
            avg_word_length = average_word_length(text)

            metrics={"avg_sentence_length":avg_sentence_length, "complex_word_count":complex_word_count, "syllable_count":syllable_count, "personal_pronouns_count":personal_pronouns_count, "avg_word_length":avg_word_length}

            return metrics
    

In [11]:
data=['https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/', 'div', 'td-post-content tagdiv-type']

data[2]=data[2].replace(" ", ".")

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    pre_cleaning = PreCleaning(config=data_ingestion_config)
    metrics = pre_cleaning.calculate(data)
except Exception as e:
    raise e

[2024-08-01 12:54:42,112: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-01 12:54:42,114: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-01 12:54:42,115: INFO: common: created directory at: artifacts]
[2024-08-01 12:54:42,116: INFO: common: created directory at: artifacts/data_ingestion]


In [13]:
metrics

{'avg_sentence_length': 17.2090395480226,
 'complex_word_count': 323,
 'syllable_count': 5681,
 'personal_pronouns_count': 2,
 'avg_word_length': 5.573539067629678}